In [ ]:
import os
import pandas as pd
import ast
import re

In [ ]:
# ottengo la lista dei file nella cartella decoding
files = os.listdir("generations")

# Estrazione dei dataframe e preprocessing
model_generations = {}
for file in files:
    # non considerare le cartelle ma solo i file csv
    if file.endswith(".csv"):
        model = re.sub(r"^|-en-decoding.csv", "", file) 
        print(model)
        model_generations[model] = pd.read_csv(f"generations/{file}")  

In [ ]:
from collections import Counter
import ast

# Funzione per calcolare precisione e recall per ogni label
def calculate_precision_recall(predictions, actuals):
    precision = {}
    recall = {}
    
    # Iteriamo su ciascun key (label) per calcolare precisione e recall
    for key in predictions.keys():
        pred_value = predictions.get(key)
        ref_value = actuals.get(key)

       
        # Se entrambi sono liste, appianiamo le liste (flatten)
        if isinstance(pred_value, list) and isinstance(ref_value, list):
            # Appianiamo le liste (flatten) se necessario
            pred_value = [item for sublist in pred_value for item in (sublist if isinstance(sublist, list) else [sublist])]
            ref_value = [item for sublist in ref_value for item in (sublist if isinstance(sublist, list) else [sublist])]
            
            #print("Prima")
            #print(pred_value)
            #print(ref_value)

            # Trasforma le tuple in liste
            pred_value = [list(item) if isinstance(item, tuple) else item for item in pred_value]
            ref_value = [list(item) if isinstance(item, tuple) else item for item in ref_value]

            # Trasforma le liste dentro le liste in stringhe
            pred_value = [str(item) if isinstance(item, list) else item for item in pred_value]
            ref_value = [str(item) if isinstance(item, list) else item for item in ref_value]

            #print("Dopo")
            #print(pred_value)
            #print(ref_value)

            #print()

            # Usa Counter per confrontare le liste
            pred_counter = Counter(pred_value)
            ref_counter = Counter(ref_value)

            # Calcola TP, FP, FN
            TP = sum((pred_counter & ref_counter).values())
            FP = sum((pred_counter - ref_counter).values())
            FN = sum((ref_counter - pred_counter).values())
        else:
            # Se non sono liste, confronta direttamente i valori
            TP = 1 if pred_value == ref_value else 0
            FP = 1 if pred_value != ref_value else 0
            FN = 1 if ref_value is not None and pred_value != ref_value else 0

        # Calcola precisione e recall per la label corrente
        precision[key] = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall[key] = TP / (TP + FN) if (TP + FN) > 0 else 0

    return precision, recall

# Per ogni modello, calcola precisione e recall per ogni label
for model in model_generations:
    print(f"Risultati per il modello '{model}':")

    # Convertiamo le colonne 'prediction' e 'actual' da stringa a dizionario
    predictions = model_generations[model]['prediction'].apply(ast.literal_eval)
    actuals = model_generations[model]['actual'].apply(ast.literal_eval)

    # Creiamo un nuovo oggetto per tenere i valori trasformati
    for i in range(len(actuals)):
        # Prendi il dizionario per ogni riga
        actual = actuals[i]
        
        # Itera sulle chiavi e valori del dizionario
        for k, v in actual.items():
            if k == 'sv' or k == 'sn':
                # Se il valore è una stringa, prova a trasformarlo in una lista
                if isinstance(v, str):
                    try:
                        # Usa ast.literal_eval per trasformare la stringa in una struttura di dati (lista o dizionario)
                        actual[k] = ast.literal_eval(v)
                    except:
                        # Se non riesci a fare la conversione, lascia il valore invariato
                        pass
                elif isinstance(v, list):
                    # Se è una lista, assicuriamoci che sia una lista piatta (flatten)
                    actual[k] = [item for sublist in v for item in (sublist if isinstance(sublist, list) else [sublist])]
    
        prediction = predictions[i]

        # Itera sulle chiavi e valori del dizionario
        for k, v in prediction.items():
            if k == 'sv' or k == 'sn':
                # Se il valore è una stringa, prova a trasformarlo in una lista
                if isinstance(v, str):
                    try:
                        # Usa ast.literal_eval per trasformare la stringa in una struttura di dati (lista o dizionario)
                        prediction[k] = ast.literal_eval(v)
                    except:
                        # Se non riesci a fare la conversione, lascia il valore invariato
                        pass
                elif isinstance(v, list):
                    # Se è una lista, assicuriamoci che sia una lista piatta (flatten)
                    prediction[k] = [item for sublist in v for item in (sublist if isinstance(sublist, list) else [sublist])]
    

    # Inizializzare variabili per tenere traccia delle metriche
    total_precision = {key: 0 for key in ['da', 'ar', 'in', 'sn', 'sv']}
    total_recall = {key: 0 for key in ['da', 'ar', 'in', 'sn', 'sv']}
    total_count = len(predictions)

    # Calcoliamo la precisione e recall per ogni esempio
    for i in range(total_count):
        prediction = predictions[i]
        actual = actuals[i]

        #print(prediction)
        #print(actual)

        precision, recall = calculate_precision_recall(prediction, actual)
        
        # Aggiorniamo le metriche cumulative, solo se la chiave è presente
        for key in total_precision.keys():
            if key in precision:  # Solo se la chiave è presente nel dizionario delle predizioni
                total_precision[key] += precision[key]
            if key in recall:  # Solo se la chiave è presente nel dizionario delle predizioni
                total_recall[key] += recall[key]

    # Calcoliamo la media delle precisioni e recall
    avg_precision = {key: total_precision[key] / total_count for key in total_precision}
    avg_recall = {key: total_recall[key] / total_count for key in total_recall}
    f1_score = {}
    for key in avg_precision:
        if (avg_precision[key] + avg_recall[key]) > 0:
            f1_score[key] = 2 * avg_precision[key] * avg_recall[key] / (avg_precision[key] + avg_recall[key])
        else:
            f1_score[key] = 0  # Se la somma di precision e recall è zero, imposta F1 a 0

    # Stampa i risultati per ogni modello
    for key in avg_precision:
        print(f"{key} - Precision: {avg_precision[key]:.2f}, Recall: {avg_recall[key]:.2f}, F1 Score: {f1_score[key]:.2f}")
    print()

In [ ]:
from collections import Counter
import ast

# Funzione per calcolare precisione e recall per ogni label
def calculate_precision_recall(predictions, actuals):
    precision = {}
    recall = {}
    
    # Iteriamo su ciascun key (label) per calcolare precisione e recall
    for key in predictions.keys():
        pred_value = predictions.get(key)
        ref_value = actuals.get(key)

        # Se entrambi sono liste, appianiamo le liste (flatten)
        if isinstance(pred_value, list) and isinstance(ref_value, list):
            pred_value = [item for sublist in pred_value for item in (sublist if isinstance(sublist, list) else [sublist])]
            ref_value = [item for sublist in ref_value for item in (sublist if isinstance(sublist, list) else [sublist])]
            
            pred_value = [list(item) if isinstance(item, tuple) else item for item in pred_value]
            ref_value = [list(item) if isinstance(item, tuple) else item for item in ref_value]

            pred_value = [str(item) if isinstance(item, list) else item for item in pred_value]
            ref_value = [str(item) if isinstance(item, list) else item for item in ref_value]

            pred_counter = Counter(pred_value)
            ref_counter = Counter(ref_value)

            TP = sum((pred_counter & ref_counter).values())
            FP = sum((pred_counter - ref_counter).values())
            FN = sum((ref_counter - pred_counter).values())
        else:
            TP = 1 if pred_value == ref_value else 0
            FP = 1 if pred_value != ref_value else 0
            FN = 1 if ref_value is not None and pred_value != ref_value else 0

        precision[key] = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall[key] = TP / (TP + FN) if (TP + FN) > 0 else 0

    return precision, recall

for model in model_generations:
    print(f"Risultati per il modello '{model}':")

    predictions = model_generations[model]['prediction'].apply(ast.literal_eval)
    actuals = model_generations[model]['actual'].apply(ast.literal_eval)

    for i in range(len(actuals)):
        actual = actuals[i]
        prediction = predictions[i]
        
        def safe_list(value):
            if isinstance(value, str):
                try:
                    return ast.literal_eval(value) if isinstance(ast.literal_eval(value), list) else [ast.literal_eval(value)]
                except:
                    return [value]  # Se fallisce la conversione, mettiamo la stringa in una lista
            return value if isinstance(value, list) else [value]  # Se non è lista, lo trasformiamo in una lista

        actual['s'] = safe_list(actual.get('sn', [])) + safe_list(actual.get('sv', []))
        prediction['s'] = safe_list(prediction.get('sn', [])) + safe_list(prediction.get('sv', []))

                
        for k, v in actual.items():
            if k in ['sn', 'sv', 's']:
                if isinstance(v, str):
                    try:
                        actual[k] = ast.literal_eval(v)
                    except:
                        pass
                elif isinstance(v, list):
                    actual[k] = [item for sublist in v for item in (sublist if isinstance(sublist, list) else [sublist])]
        
        for k, v in prediction.items():
            if k in ['sn', 'sv', 's']:
                if isinstance(v, str):
                    try:
                        prediction[k] = ast.literal_eval(v)
                    except:
                        pass
                elif isinstance(v, list):
                    prediction[k] = [item for sublist in v for item in (sublist if isinstance(sublist, list) else [sublist])]
    
    total_precision = {key: 0 for key in ['da', 'ar', 'in', 'sn', 'sv', 's']}
    total_recall = {key: 0 for key in ['da', 'ar', 'in', 'sn', 'sv', 's']}
    total_count = len(predictions)

    for i in range(total_count):
        prediction = predictions[i]
        actual = actuals[i]

        precision, recall = calculate_precision_recall(prediction, actual)
        
        for key in total_precision.keys():
            if key in precision:
                total_precision[key] += precision[key]
            if key in recall:
                total_recall[key] += recall[key]

    avg_precision = {key: total_precision[key] / total_count for key in total_precision}
    avg_recall = {key: total_recall[key] / total_count for key in total_recall}
    f1_score = {}
    for key in avg_precision:
        if (avg_precision[key] + avg_recall[key]) > 0:
            f1_score[key] = 2 * avg_precision[key] * avg_recall[key] / (avg_precision[key] + avg_recall[key])
        else:
            f1_score[key] = 0

    for key in avg_precision:
        print(f"{key} - Precision: {avg_precision[key]:.2f}, Recall: {avg_recall[key]:.2f}, F1 Score: {f1_score[key]:.2f}")
    print()
